In this example we will do the interesting stuff - we will watch videos from different video platforms and capturing the corresponding traffic.

Watching video is a pretty popular task, used for Quality of Experience research, Variable Bit Rate experiments, traffic-based video classification, etc. Let's collect a toy dataset using netunicorn platform for these tasks.

As usual, we need to import all needed classes and tasks. This time we are again using our beautiful netunicorn-library and preimplemented tasks for watching YouTube, Vimeo, and Twitch videos. Please, visit the corresponding GitHub repository (https://github.com/netunicorn/netunicorn-library) if you're interested in tasks details or want to contribute your tasks and pipelines.

_Important:_ for this example, we assume that you already explored the basic example and familiar with netunicorn principles.

In [1]:
import os
import time

from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline

# Tasks to start tcpdump and stop named tcpdump task
from netunicorn.library.tasks.capture.tcpdump import StartCapture, StopNamedCapture

# Upload to file.io - public anonymous temporary file storage
from netunicorn.library.tasks.upload.fileio import UploadToFileIO

# Tasks for watching the corresponding video platform
from netunicorn.library.tasks.video_watchers.youtube_watcher import WatchYouTubeVideo
from netunicorn.library.tasks.video_watchers.vimeo_watcher import WatchVimeoVideo
from netunicorn.library.tasks.video_watchers.twitch_watcher import WatchTwitchStream

As the next step, let's assemble our pipeline. We should start the traffic capture, then watch three absolutely random videos (or streams) from YouTube, Vimeo, and Twitch correspondingly, and then again stop the data capture and upload the resulting PCAP file to some cloud. We again will use `file.io` for data upload.

In [2]:
pipeline = (
    Pipeline()
    .then(StartCapture(filepath="/tmp/capture.pcap", name="capture"))
    .then(WatchYouTubeVideo("https://www.youtube.com/watch?v=dQw4w9WgXcQ", 10))
    .then(WatchVimeoVideo("https://vimeo.com/375468729", 10))
    .then(WatchTwitchStream("https://www.twitch.tv/shroud", 10))
    .then(StopNamedCapture(start_capture_task_name="capture"))
    .then(UploadToFileIO(filepath="/tmp/capture.pcap", expires="1d"))
)

Most of the other code is not different from any experiment definition. Let's use correct credentials for our infrastructure (in your case they could be different, if you don't have netunicorn instance in your organization - you can deploy your own locally for testing purposes, see https://netunicorn.cs.ucsb.edu/examples for details)

In [3]:
# if you have .env file locally for storing credentials, skip otherwise
if '.env' in os.listdir():
    from dotenv import load_dotenv
    load_dotenv(".env")

NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'http://localhost:26611')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', 'test')
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', 'test')

client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
client.healthcheck()

True

Let's get some nodes for execution. As usual, for demonstration purposes of this notebook we will take our Raspberry Pi nodes, but if your infrastructure is different - feel free to modify the next cell.

In [4]:
nodes = client.get_nodes()

# switch for showing our infrastructure vs you doing it locally on other nodes
if os.environ.get('NETUNICORN_ENDPOINT', 'http://localhost:26611') != 'http://localhost:26611':
    working_nodes = nodes.filter(lambda node: node.name.startswith("raspi")).take(5)
else:
    working_nodes = nodes.take(1)

working_nodes

[raspi-e4:5f:01:72:a2:eb,
 raspi-e4:5f:01:56:d6:ce,
 raspi-e4:5f:01:75:6b:2c,
 raspi-e4:5f:01:a0:32:5f,
 raspi-e4:5f:01:72:89:99]

In [5]:
# Creating the experiment
experiment = Experiment().map(pipeline, working_nodes)
experiment

 - Deployment: Node=raspi-e4:5f:01:72:a2:eb, executor_id=, prepared=False
 - Deployment: Node=raspi-e4:5f:01:56:d6:ce, executor_id=, prepared=False
 - Deployment: Node=raspi-e4:5f:01:75:6b:2c, executor_id=, prepared=False
 - Deployment: Node=raspi-e4:5f:01:a0:32:5f, executor_id=, prepared=False
 - Deployment: Node=raspi-e4:5f:01:72:89:99, executor_id=, prepared=False

When pipelines are assigned to nodes (and therefore, Deployments are created), each pipeline would have an "environment_definition" with a list of "preparation" commands to execute during Docker image compilation. We can explore these commands:

In [6]:
for line in experiment[0].environment_definition.commands:
    print(line)

sudo apt-get update
sudo apt-get install -y tcpdump
apt update
apt install -y python3-pip wget xvfb chromium
pip3 install selenium webdriver-manager
python -c "from webdriver_manager.chrome import ChromeDriverManager; from webdriver_manager.core.utils import ChromeType; ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()"
ln -s /root/.wdm/drivers/chromedriver /usr/bin/chromedriver
apt update
apt install -y python3-pip wget xvfb chromium
pip3 install selenium webdriver-manager
python -c "from webdriver_manager.chrome import ChromeDriverManager; from webdriver_manager.core.utils import ChromeType; ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()"
ln -s /root/.wdm/drivers/chromedriver /usr/bin/chromedriver
apt update
apt install -y python3-pip wget xvfb chromium
pip3 install selenium webdriver-manager
python -c "from webdriver_manager.chrome import ChromeDriverManager; from webdriver_manager.core.utils import ChromeType; ChromeDriverManager(chrome_type=ChromeType.

Seems like we can optimize them a bit, as all watcher tasks use the same commands for configuration (at least, for now). Remember, that tasks do not know about each other in the pipeline, so they cannot resolve these duplications automatically.

But, instead, let's ask experiment to use a predefined Docker image. This image was created by taking a base `python:3.10.9-slim` image and installing all the programs from the current environment. You can create and optimize your own image and use it on netunicorn platform as well, as soon as it's published on any public Docker repository (e.g., DockerHub).

Please, note that any image used for the platform should have `netunicorn-executor` installed and `python3 -m netunicorn.executor` as the entrypoint.

In [7]:
from netunicorn.base import DockerImage
for deployment in experiment:
    # you can explore the image on the DockerHub
    deployment.environment_definition = DockerImage(image='netunicorn/chromium:0.3.0')

Ok, not let's define the experiment name and, as usual, prepare the experiment.

In [8]:
experiment_label = "video_watchers_example"

In [9]:
try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

client.prepare_experiment(experiment, experiment_label)

while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status == ExperimentStatus.READY:
        break
    time.sleep(20)

ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.READY


Verifying that everything is correct:

In [10]:
for deployment in client.get_experiment_status(experiment_label).experiment:
    print(f"Prepared: {deployment.prepared}, error: {deployment.error}")

Prepared: True, error: None
Prepared: True, error: None
Prepared: True, error: None
Prepared: True, error: None
Prepared: True, error: None


And starting the execution:

In [11]:
client.start_execution(experiment_label)

while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status != ExperimentStatus.RUNNING:
        break
    time.sleep(20)

ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.FINISHED


Moment of truth... /nervous drums sounds/...

In [12]:
from returns.pipeline import is_successful
from returns.result import Result

for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")

    result, log = report.result  # report stores results of execution and corresponding log
    
    # result is a returns.result.Result object, could be Success of Failure
    # or None is error occured during execution
    print(f"Result is: {type(result)}")
    if isinstance(result, Result):
        data = result.unwrap() if is_successful(result) else result.failure()
        for key, value in data.items():
            print(f"{key}: {value}")

    # we also can explore logs
    for line in log:
        print(line.strip())
    print()

Node name: raspi-e4:5f:01:72:a2:eb
Error: None
Result is: <class 'returns.result.Success'>
capture: [<Success: 11>]
04071c7c-bfd1-4132-b8d5-f34d4fa5a730: [<Success: Video finished by timeout: 10 seconds>]
d3731da5-a7aa-470e-816d-bfd11942df19: [<Success: Video finished by timeout: 10 seconds>]
195d48cf-49f5-48db-926a-879024c01fe8: [<Success: Video probably finished by timeout: 10 seconds>]
cdaac4b5-cc82-494f-a12a-56f66a4a19cd: [<Success: b''>]
3ade0be7-f9e1-40fd-bc21-73b61287d7cb: [<Success: {"success":true,"status":200,"id":"96981ef0-07de-11ee-afaa-71b35325ba38","key":"tSEh0v3Cm8vl","path":"/","nodeType":"file","name":"capture.pcap","title":null,"description":null,"size":48020619,"link":"https://file.io/tSEh0v3Cm8vl","private":false,"expires":"2023-06-11T22:31:48.564Z","downloads":0,"maxDownloads":1,"autoDelete":true,"planId":0,"screeningStatus":"pending","mimeType":"application/octet-stream","created":"2023-06-10T22:31:48.564Z","modified":"2023-06-10T22:31:48.564Z"}>]
Parsed configura

Well, what could go wrong? :) ~(everything...)~  
We successfully executed the experiment and watched 10 seconds of videos from YouTube, Vimeo, and Twitch, and recorded the network traffic for further analysis. Well, most likely it's the ad traffic from the corresponding platforms (as we intentionally did not used ad blockers in this example), but we did it anyway! :D  

We encourage everyone to implement corresponding tasks or tasks modifications as you need and publish them to [netunicorn-library](https://github.com/netunicorn/netunicorn-library). As usual, our documenation and other examples are available on https://netunicorn.cs.ucsb.edu.